You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [3]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

  Obtaining dependency information for crewai==0.28.8 from https://files.pythonhosted.org/packages/d2/78/dd949ea0d914ce306821067a886b3520755b59df71cd868a2997cc90a7a5/crewai-0.28.8-py3-none-any.whl.metadata
  Obtaining dependency information for crewai_tools==0.1.6 from https://files.pythonhosted.org/packages/de/35/7b0ab9c9de202d2fc22dc183ec9da28c992038938f1e06684f72ba15eb55/crewai_tools-0.1.6-py3-none-any.whl.metadata
  Obtaining dependency information for langchain_community==0.0.29 from https://files.pythonhosted.org/packages/e1/80/4136757bf245f60e2f4e148adb340207ea330be92bc26727bca32fe6bc48/langchain_community-0.0.29-py3-none-any.whl.metadata
  Obtaining dependency information for embedchain<0.2.0,>=0.1.98 from https://files.pythonhosted.org/packages/fb/57/c9294d3f7eae142f6dfe77eda6e1541923c7ad5ebf389cb07f9ada6c53a2/embedchain-0.1.126-py3-none-any.whl.metadata
  Obtaining dependency information for instructor<0.6.0,>=0.5.2 from https://files.pythonhosted.org/packages/3d/57/35e22db00

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')
from cryptography.utils import CryptographyDeprecationWarning

warnings.filterwarnings("ignore", category=CryptographyDeprecationWarning)

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from dotenv import load_dotenv
import httpx
import openai

# Load environment variables
load_dotenv()

# Get the OpenAI API key from the environment variables
openai_api_key = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'




## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


# Agent: Content Planner
## Final Answer: 
Title: The Evolution of Artificial Intelligence: Trends, Players, and News

Outline:
I. Introduction
    A. Definition of Artificial Intelligence
    B. Importance of AI in today’s world
II. Latest Trends in Artificial Intelligence
    A. Machine Learning advancements
    B. Natural Language Processing innovations
    C. Robotics applications
III. Key Players in the AI Industry
    A. Tech giants like Google, Microsoft, and Amazon
    B. AI startups disrupting the industry
IV. Noteworthy News in Artificial Intelligence
    A. Breakthrough research studies
    B. Industry collaborations and

- Display the results of your execution as markdown in the notebook.

In [15]:

# Modify the display code to access the raw string content
from IPython.display import Markdown
Markdown(result.raw)  # Access the .raw attribute of the CrewOutput object

# Unleashing the Power of Crew AI for Patent Drafting: Best Practices Unveiled

## Introduction

In the ever-changing landscape of intellectual property, artificial intelligence (AI) has made a significant impact on various processes, particularly patent drafting. Crew AI, an innovative AI tool, has become well-known for its efficiency and effectiveness in supporting patent professionals and innovators in creating high-quality patents. By harnessing the capabilities of Crew AI, individuals can boost their productivity and accuracy, leading to the development of stronger patent applications.

## Understanding the Target Audience

The target audience for utilizing Crew AI in patent drafting includes patent attorneys, patent agents, and inventors or innovators who operate within the intricate realm of intellectual property. These individuals aim to navigate the complexities of patent drafting with precision and ingenuity. By comprehending the unique requirements and obstacles faced by this audience, we can tailor our approach to offer the most valuable insights and recommendations.

## Best Practices for Using Crew AI in Patent Drafting

### Setting Clear Objectives

Prior to commencing patent drafting with Crew AI, it is imperative to define clear objectives for the process. Establishing the scope of the patent application, identifying key innovative aspects to emphasize, and outlining the technical details to be included are essential steps. Precise goal-setting will enable the AI tool to effectively aid in achieving the desired outcomes.

### Understanding Patent Requirements

Successful patent drafting demands a deep understanding of the legal and technical components involved. Familiarizing oneself with the specific requirements of patent applications, such as patentability criteria, disclosure requirements, and formalities, is crucial. While Crew AI can assist in interpreting intricate patent laws and guidelines, a solid knowledge base in patent drafting is fundamental for optimal results.

### Leveraging AI Features

Exploit the diverse range of AI features provided by Crew AI to maximize its potential in patent drafting. Make use of functionalities like natural language processing for formulating patent claims, semantic analysis for improving description clarity, and automated review tools for detecting errors. Integrating these AI capabilities into your workflow can significantly streamline the drafting process.

### Ensuring Accuracy and Compliance

Despite the powerful capabilities of AI tools in patent drafting, it is vital to verify the accuracy and compliance of the content produced. Regularly reviewing the output generated by Crew AI to confirm the correctness of technical descriptions, claims, and specifications is essential. Adhering to patent regulations and guidelines is imperative to uphold the authenticity and integrity of the patent application.

### Reviewing and Polishing the Draft

A crucial final step in leveraging Crew AI for patent drafting involves thorough review and refinement of the draft. Engage in manual editing and enhancement to elevate the clarity, coherence, and conciseness of the patent application. Human intervention plays a pivotal role in refining the output generated by AI, ensuring that the final draft meets the highest standards of quality and comprehensiveness.

## Call to Action

To delve deeper into the capabilities of Crew AI in patent drafting, we urge readers to experience the efficiency and innovation it brings to the process. Embrace the future of patent drafting tools by integrating Crew AI into your workflow. For additional guidance and support in utilizing AI for patents, do not hesitate to reach out to our team for personalized assistance.

## SEO Keywords
Crew AI, patent drafting, artificial intelligence in patents, patent writing tools, patent attorney software

## Resources
- **USPTO**: United States Patent and Trademark Office
- **WIPO**: World Intellectual Property Organization
- **IAM**: Intellectual Asset Management

In conclusion, the utilization of Crew AI in patent drafting offers a significant opportunity for patent professionals and innovators to enhance their efficiency and precision in creating high-quality patent applications. By adhering to the best practices outlined above and capitalizing on the advanced capabilities of Crew AI, individuals can streamline their patent drafting process and enhance the quality of their patent submissions. Embrace the power of AI in patents and embark on a journey towards greater innovation and success.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [13]:
topic = "Best practices for using Crew AI to write patents"
result = crew.kickoff(inputs={"topic": topic})

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Best practices for using Crew AI to write patents.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


# Agent: Content Planner
## Final Answer: 
Title: Unleashing the Power of Crew AI for Patent Drafting: Best Practices Unveiled

Outline:
I. Introduction
    A. Definition of Crew AI
    B. Importance of Crew AI in patent drafting
II. Understanding the Target Audience
    A. Patent Attorneys 
    B. Patent Agents 
    C. Inventors and Innovators 
III. Best Practices for Using Crew AI in Patent Drafting
    A. Setting Clear Objectives 
    B. Understanding Patent Requirements 
    C. Leveraging AI Features 
    D. Ensuring Accuracy and Compliance 
    E. Reviewing and Polishing the Draft 
IV. Call to Action


In [17]:
Markdown(result.raw)

# Unleashing the Power of Crew AI for Patent Drafting: Best Practices Unveiled

## Introduction

In the ever-changing landscape of intellectual property, artificial intelligence (AI) has made a significant impact on various processes, particularly patent drafting. Crew AI, an innovative AI tool, has become well-known for its efficiency and effectiveness in supporting patent professionals and innovators in creating high-quality patents. By harnessing the capabilities of Crew AI, individuals can boost their productivity and accuracy, leading to the development of stronger patent applications.

## Understanding the Target Audience

The target audience for utilizing Crew AI in patent drafting includes patent attorneys, patent agents, and inventors or innovators who operate within the intricate realm of intellectual property. These individuals aim to navigate the complexities of patent drafting with precision and ingenuity. By comprehending the unique requirements and obstacles faced by this audience, we can tailor our approach to offer the most valuable insights and recommendations.

## Best Practices for Using Crew AI in Patent Drafting

### Setting Clear Objectives

Prior to commencing patent drafting with Crew AI, it is imperative to define clear objectives for the process. Establishing the scope of the patent application, identifying key innovative aspects to emphasize, and outlining the technical details to be included are essential steps. Precise goal-setting will enable the AI tool to effectively aid in achieving the desired outcomes.

### Understanding Patent Requirements

Successful patent drafting demands a deep understanding of the legal and technical components involved. Familiarizing oneself with the specific requirements of patent applications, such as patentability criteria, disclosure requirements, and formalities, is crucial. While Crew AI can assist in interpreting intricate patent laws and guidelines, a solid knowledge base in patent drafting is fundamental for optimal results.

### Leveraging AI Features

Exploit the diverse range of AI features provided by Crew AI to maximize its potential in patent drafting. Make use of functionalities like natural language processing for formulating patent claims, semantic analysis for improving description clarity, and automated review tools for detecting errors. Integrating these AI capabilities into your workflow can significantly streamline the drafting process.

### Ensuring Accuracy and Compliance

Despite the powerful capabilities of AI tools in patent drafting, it is vital to verify the accuracy and compliance of the content produced. Regularly reviewing the output generated by Crew AI to confirm the correctness of technical descriptions, claims, and specifications is essential. Adhering to patent regulations and guidelines is imperative to uphold the authenticity and integrity of the patent application.

### Reviewing and Polishing the Draft

A crucial final step in leveraging Crew AI for patent drafting involves thorough review and refinement of the draft. Engage in manual editing and enhancement to elevate the clarity, coherence, and conciseness of the patent application. Human intervention plays a pivotal role in refining the output generated by AI, ensuring that the final draft meets the highest standards of quality and comprehensiveness.

## Call to Action

To delve deeper into the capabilities of Crew AI in patent drafting, we urge readers to experience the efficiency and innovation it brings to the process. Embrace the future of patent drafting tools by integrating Crew AI into your workflow. For additional guidance and support in utilizing AI for patents, do not hesitate to reach out to our team for personalized assistance.

## SEO Keywords
Crew AI, patent drafting, artificial intelligence in patents, patent writing tools, patent attorney software

## Resources
- **USPTO**: United States Patent and Trademark Office
- **WIPO**: World Intellectual Property Organization
- **IAM**: Intellectual Asset Management

In conclusion, the utilization of Crew AI in patent drafting offers a significant opportunity for patent professionals and innovators to enhance their efficiency and precision in creating high-quality patent applications. By adhering to the best practices outlined above and capitalizing on the advanced capabilities of Crew AI, individuals can streamline their patent drafting process and enhance the quality of their patent submissions. Embrace the power of AI in patents and embark on a journey towards greater innovation and success.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).